# Description

The small "library" of functions on integers in the setup perform some simple number theoretic or combinatorics operations.  However, they are little documented and each has a somewhat dense implementation.  For this exercise, you should write a single `unittest` class called `TestLibrary`.  In the exercise for that last lesson, you wrote a number of doctests; testing the same features is acceptable, but other tests may lend themselves better to a `unittest` approach.

The exercises in this courses have a large space of possible solutions.  The "solution" provided gives plausible tests to include, but yours may well be substantially different and yet no less useful or relevant.  Your solution might well be better than that suggested.

Some of these functions deal with prime numbers.  To aid in your exploration or tests, the file `primes-5000.txt` in this repository contains the first 5000 prime numbers.  These numbers *are correct* and you may use them as a reference in whatever manner you find useful.

# Setup

In [1]:
import unittest
from math import sqrt, log

def get_primes_upto(limit):
    "A list of all primes less than or equal to limit"
    is_prime = [False] * 2 + [True] * (limit-1)
    for n in range(int(sqrt(limit) + 1.5)): 
        if is_prime[n]:
            is_prime[n**2::n] = [False] * ((limit - n**2)//n + 1)
    return trues(is_prime)

def prime_count(limit):
    "Upper bound on number of primes below a limit"
    # Gauss/Legendre approx, padded to exceed π(x) for small limits
    return int(1.2 * limit/log(limit))

def get_init_primes(N):
    "Return the first N prime numbers"
    # Find "enough" primes
    limit = 8
    while N > prime_count(limit):
        limit *= 2
    many_primes = get_primes_upto(limit)
    # Return exactly N of them
    return many_primes[:N]

def trues(it): 
    "Which elements of 'bitfield iterable' are True?"
    return [n for n, target in enumerate(it) if target]

def sums_of_subset(numbers):
    "The natural numbers that are sums of subsets of initial set"
    numbers = sorted(numbers)                 # Numbers in ascending order
    sum_of_numbers = sum(numbers)
    reachable = [False] * (sum_of_numbers+1)  # Trues as one-based index
    for p in numbers:
        reachable[p] = True 
        for n, target in enumerate(reachable[:]):
            if target and  n != p:
                reachable[p+n] = True
    return trues(reachable)

def pair_sums(numbers, allow_doubles=False):
    "Sums of elements from initial set"
    sums = set()
    numbers = sorted(numbers)
    for i in numbers:
        for j in numbers:
            if allow_doubles or i != j:
                sums.add(i+j)
    return sums

# Solution

In [2]:
class TestLibrary(unittest.TestCase):
    def setUp(self):
        self.primes_5k = [int(n) for n in open('primes-5000.txt')]
        self.prime1000 = prime_count(1000)
        
    def test_prime_generation(self):
        # Top is not prime
        self.assertEqual(get_primes_upto(100)[-1], 97)
        # Top is prime
        self.assertEqual(get_primes_upto(101)[-1], 101)
    
    def test_prime_accuracy(self):
        # Check some against reference
        self.assertEqual(get_init_primes(1000), self.primes_5k[:1000])
        
    def test_prime_count(self):
        # 168 is exact π(1000) value
        self.assertTrue(200 >= self.prime1000 >= 168)
        self.assertTrue(prime_count(1e10) >= 455_052_511)

    def test_trues(self):
        bools = [True, True, False, False, True, False, True]
        self.assertTrue(trues(bools), [0, 1, 4, 6])
    
    def test_pair_sums(self):
        self.assertTrue(pair_sums([3, 4, 5]), {7, 8, 9})
        
    def test_pair_sums_dups(self):
        self.assertTrue(pair_sums([3, 5], allow_doubles=True), {6, 8, 10})
        
    def test_Goldbach(self):
        twoprime = pair_sums(get_init_primes(7), True)
        for even in range(4, 32, 2):
            self.assertIn(even, twoprime)
        
    def test_unreachables(self):
        # The only small numbers not sum of primes
        unreachable = {1, 4, 6}
        sums = set(sums_of_subset(get_init_primes(400)))
        self.assertEqual({i for i in range(1, 10_000) if i not in sums}, 
                         unreachable)

# Test Cases

In [3]:
def test_is_test_class():
    assert issubclass(TestLibrary, unittest.TestCase)
    
test_is_test_class()

In [4]:
def test_all():
    runner = TestLibrary()
    runner.setUp()
    tests = [getattr(runner, name) for name in dir(runner) 
             if name.startswith('test_')]
    assert len(tests) >= 8, "At least 8 tests are required"
    for test in tests:
        test()

test_all()